# Сбор данных

Начнём первый этап - полезем за данными. Для этого мы взяли сайт Дром.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_bikes(url):
    res = requests.get(url)
    tree = BeautifulSoup(res.content, 'html.parser')
    return tree

In [3]:
data = []

In [4]:
for p in range(1, 50):
  url = f"https://moto.drom.ru/sale/?page={p}"
  tree = get_bikes(url)
  bikes_cards = tree.find_all('tr', class_='bull-list-item-js -exact')
  for j in range(len(bikes_cards)):
    title_name = bikes_cards[j].find('a', class_='bulletinLink bull-item__self-link auto-shy')
    price_bike = bikes_cards[j].find('div', class_='price-block__final-price finalPrice')
    tech = bikes_cards[j].find('div', class_='bull-item__annotation-row')
    city = bikes_cards[j].find('div', class_='bull-delivery bull-delivery_city')

    if title_name:
      if price_bike:
        if tech:
          if city:
            data.append([title_name.text.strip(), price_bike.get_text(), tech.text, city.text])

In [5]:
len(data)

1085

После парсинга имеем массив в 1085 байков. Теперь запишем полученные данные в csv файл, чтобы можно было с ними работать.

In [6]:
import csv

In [7]:
with open('/content/drive/MyDrive/ProjectA/data_for_bikes.csv', 'w', encoding="utf-8") as csvfile: #код для записи в csv
    csvwriter = csv.writer(csvfile)
    for row in data:
        csvwriter.writerow(row)

Код писался в колабе, поэтому csv файл создавался на гугл диске. Приклеплю его на гитхабе отдельно.